In [42]:
from matplotlib import pyplot as plt
import pandas as pd

In [43]:
import matplotlib
matplotlib.rcParams['text.usetex'] = True
plt.rcParams['legend.title_fontsize'] = 'xx-small'

In [44]:
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "sans-serif",
#     "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
# plt.rcParams.update({
#     "text.usetex": True,
#     "font.family": "serif",
#     "font.serif": ["Palatino"],
# })

In [45]:
def error_cal(data, duration):
    
    df = data[data['duration'] == duration].copy()

    df['l_err'] = abs(df['min'] - df['mean'])
    df['u_err'] = abs(df['max'] - df['mean'])

    yerr = [df['l_err'].tolist(), df['u_err'].tolist()]
    
    return df, yerr

In [118]:
def plot(fixed, flow):
    data = fixed[fixed['flow'] == flow].drop(columns=['flow'])

    data = data.groupby(['duration', 'cap']).agg({'result':['mean', 'min', 'max']})

    data.columns = data.columns.droplevel()
    data = data.reset_index()
    
    fig, ax = plt.subplots(figsize=(9,6))

    for duration in [0.2, 0.25, 0.33, 0.5]:
        duration_df, yerr = error_cal(data, duration)
        ax.errorbar(duration_df['cap'], duration_df['mean'], yerr=yerr, capsize=8, label='{}'.format(duration))
        
    ax.set_xlim([0.5, 9.5])
    
    ax.set_xlabel(r'Initially allocated CV capacity (cv\_ini\_cap)', fontsize=16)
    ax.set_ylabel(r'Performance (negative total reroutes)', fontsize=16)
    ax.legend(title="parking duration ratio\n (cv2ncv\_pd)", fontsize=14, title_fontsize=16, loc='best') 
    # bbox_to_anchor=(0.8,0.5)
    ax.tick_params(axis='both', which='major', labelsize=14)

    ax.get_xaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x))))
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

    plt.savefig('mp_{}.pdf'.format(flow))
    plt.close()

### fix baseline

In [111]:
data = pd.read_csv('fix_baseline.txt', delimiter = ",")

In [112]:
data = data[data['result'] > -3000]

In [113]:
for rate in [1.0, 2.0, 3.0, 4.0, 5.0]:
    plot(data, rate)

### max pressure baseline

In [119]:
data = pd.read_csv('mp_baseline.txt', delimiter = ",")

In [120]:
data = data[data['result'] > -3000]

In [121]:
for rate in [0.2, 0.25, 0.33, 0.5, 1, 2, 3, 4, 5]:
    plot(data, rate)